<a href="https://colab.research.google.com/github/27-sahithya/Adaptivecruisecontrol/blob/main/Adaptive_cruise_control.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================================
# STEP 1: Install and Prepare
# ==========================================
import cv2
import numpy as np
from google.colab import files
import os
from IPython.display import display
from base64 import b64encode

# Step 1.1 - Download the vehicle detection cascade file
!wget https://github.com/andrewssobral/vehicle_detection_haarcascades/raw/master/cars.xml -O cars.xml

# Step 1.2 - Verify that the cascade file exists
cascade_path = "cars.xml"
if not os.path.exists(cascade_path):
    raise FileNotFoundError("Cascade file not found. Download step failed!")

# Step 1.3 - Load the cascade classifier
vehicle_cascade = cv2.CascadeClassifier(cascade_path)
if vehicle_cascade.empty():
    raise RuntimeError("Failed to load the cascade file properly!")

print("✅ Vehicle cascade loaded successfully!")

# ==========================================
# STEP 2: Adaptive Cruise Control (ACC)
# ==========================================
class AdaptiveCruiseControl:
    def __init__(self, target_speed=80.0, min_distance=80.0, max_deceleration=8.0, max_acceleration=5.0):
        """
        target_speed: desired cruising speed in km/h
        min_distance: minimum safe distance (pixels, approximate for video)
        max_deceleration: maximum speed decrease per second (km/h)
        max_acceleration: maximum speed increase per second (km/h)
        """
        self.target_speed = target_speed
        self.min_distance = min_distance
        self.max_deceleration = max_deceleration
        self.max_acceleration = max_acceleration

    def update_speed(self, current_speed, distance_to_vehicle, dt):
        """
        Adjust speed based on detected front vehicle distance.
        """
        if distance_to_vehicle is not None and distance_to_vehicle < self.min_distance:
            # Too close -> slow down
            new_speed = max(0, current_speed - self.max_deceleration * dt)
        else:
            # Safe or no vehicle -> speed up toward target
            if current_speed < self.target_speed:
                new_speed = min(self.target_speed, current_speed + self.max_acceleration * dt)
            else:
                new_speed = self.target_speed
        return new_speed

# Initialize ACC
acc_system = AdaptiveCruiseControl(target_speed=80.0, min_distance=80.0)

# ==========================================
# STEP 3: Vehicle Detection
# ==========================================
def detect_vehicles(frame):
    """
    Detect vehicles and return bounding boxes + distance approximation.
    """
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    cars = vehicle_cascade.detectMultiScale(
        gray,
        scaleFactor=1.1,
        minNeighbors=3,
        minSize=(50, 50)
    )
    return cars

# ==========================================
# STEP 4: Process video with ACC
# ==========================================
def process_video_with_acc(input_video, output_video="processed_output.mp4"):
    """
    Process the video:
    - Detect cars
    - Adjust speed using ACC
    - Save processed video with visual indicators
    """
    cap = cv2.VideoCapture(input_video)
    if not cap.isOpened():
        print("❌ Error: Could not open video.")
        return None

    # Get video properties
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    # Define VideoWriter for output
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video, fourcc, fps, (width, height))

    current_speed = 50.0  # Start at 50 km/h
    frame_count = 0

    print("🎥 Processing video with Adaptive Cruise Control...")

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        dt = 1.0 / fps  # time between frames in seconds

        # Detect vehicles
        cars = detect_vehicles(frame)
        distance_to_front = None

        # Draw bounding boxes and find closest car
        for (x, y, w, h) in cars:
            cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
            # Approximate distance: larger box = closer car
            approx_distance = max(10, 200 - w)  # Avoid negative distance
            if distance_to_front is None or approx_distance < distance_to_front:
                distance_to_front = approx_distance

        # Update speed based on ACC
        current_speed = acc_system.update_speed(current_speed, distance_to_front, dt)

        # Draw speed and distance info
        cv2.putText(frame, f"Speed: {current_speed:.1f} km/h",
                    (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        if distance_to_front is not None:
            cv2.putText(frame, f"Front Vehicle Dist: {distance_to_front:.1f}",
                        (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)
        else:
            cv2.putText(frame, "Front Vehicle Dist: None",
                        (10, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 255), 2)

        # Add status text
        if distance_to_front is not None and distance_to_front < acc_system.min_distance:
            cv2.putText(frame, "SLOWING DOWN!", (width - 250, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        elif current_speed < acc_system.target_speed:
            cv2.putText(frame, "ACCELERATING", (width - 250, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 0), 2)
        else:
            cv2.putText(frame, "CRUISING", (width - 250, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

        # Write frame to output video
        out.write(frame)
        frame_count += 1

        if frame_count % 30 == 0:
            print(f"Processed {frame_count} frames...")

    cap.release()
    out.release()
    print(f"✅ Processing completed. Output saved as: {output_video}")
    return output_video

# ==========================================
# STEP 5: Upload and Run
# ==========================================
print("\nDemo Options:")
print("1. Upload a video file with ACC processing")
print("2. Exit")

choice = input("\nEnter choice (1/2): ").strip()

if choice == "1":
    print("📂 Please upload a video file from your computer...")
    uploaded = files.upload()  # Opens a file picker in Colab

    for fn in uploaded.keys():
        video_path = fn
        print(f"\nProcessing video: {video_path}")
        output_file = process_video_with_acc(video_path, output_video="processed_output.mp4")

        if output_file:
            print("\n📥 Preparing download link for the processed video...")
            files.download(output_file)

elif choice == "2":
    print("Exiting...")

else:
    print("❌ Invalid choice. Please run again.")


--2025-09-16 09:04:43--  https://github.com/andrewssobral/vehicle_detection_haarcascades/raw/master/cars.xml
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/andrewssobral/vehicle_detection_haarcascades/master/cars.xml [following]
--2025-09-16 09:04:44--  https://raw.githubusercontent.com/andrewssobral/vehicle_detection_haarcascades/master/cars.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118803 (116K) [text/plain]
Saving to: ‘cars.xml’

cars.xml            100%[===================>] 116.02K  --.-KB/s    in 0.1s    

2025-09-16 09:04:45 (957 KB/s) - ‘cars.xml’ saved [118803/118803]

✅ Ve

Saving Syakila Salsabila -Via KLICKPIN CF.mp4 to Syakila Salsabila -Via KLICKPIN CF.mp4

Processing video: Syakila Salsabila -Via KLICKPIN CF.mp4
🎥 Processing video with Adaptive Cruise Control...
Processed 30 frames...
Processed 60 frames...
Processed 90 frames...
Processed 120 frames...
Processed 150 frames...
Processed 180 frames...
Processed 210 frames...
Processed 240 frames...
Processed 270 frames...
Processed 300 frames...
Processed 330 frames...
Processed 360 frames...
Processed 390 frames...
Processed 420 frames...
Processed 450 frames...
Processed 480 frames...
Processed 510 frames...
Processed 540 frames...
Processed 570 frames...
Processed 600 frames...
Processed 630 frames...
Processed 660 frames...
Processed 690 frames...
Processed 720 frames...
Processed 750 frames...
Processed 780 frames...
Processed 810 frames...
Processed 840 frames...
Processed 870 frames...
Processed 900 frames...
Processed 930 frames...
Processed 960 frames...
Processed 990 frames...
Processed 1020

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>